In [386]:
import pandas as pd
from datetime import datetime
import os,re
import json
import statistics
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from sklearn.metrics import roc_auc_score,accuracy_score
import numpy as np
from tqdm import tqdm
from scipy import stats
from sklearn.model_selection import train_test_split

In [167]:

train=pd.read_csv('train.csv')
campaign=pd.read_csv('campaign_data.csv')
ci_mapping=pd.read_csv('coupon_item_mapping.csv')
cust_demo=pd.read_csv('customer_demographics.csv')
ct_data=pd.read_csv('customer_transaction_data.csv')
item_data=pd.read_csv('item_data.csv')

* Before joining the datasets try to fill the null values in customer demographic dataset

In [143]:
cust_demo.dtypes

customer_id        int64
age_range         object
marital_status    object
rented             int64
family_size       object
no_of_children    object
income_bracket     int64
dtype: object

#### steps:
* Left join-> train & cust_demo
* Left join-> train & campaign_data
* Inner join-> couponitem & item data,
  * aggregate over 3 on coupon_id
* Inner join train and 3-1 on coupon_id
* Inner join-> cust_trans_data & item data
  *  aggregate on customer id
* left join-> train & 5-1 on customer id

In [168]:
for col in cust_demo.columns:
    print("{} has {} null values".format(col,cust_demo[col].isnull().sum()))

customer_id has 0 null values
age_range has 0 null values
marital_status has 329 null values
rented has 0 null values
family_size has 0 null values
no_of_children has 538 null values
income_bracket has 0 null values


* As you can see that no_of_children & marital_status features has more null values than the other fields, try to fill these fields based on the other fields. 
* I strongly believe that these two fields are highly dependent on family_size as if the family_size is 1 then there is high chance that he is single and no of children 0

In [169]:
cust_demo.loc[(cust_demo['no_of_children'].isnull()) & (cust_demo['family_size'] == '1')][['no_of_children','family_size']]

,no_of_children,family_size
4,NaN,1
7,NaN,1
10,NaN,1
11,NaN,1
15,NaN,1
29,NaN,1
31,NaN,1
32,NaN,1
33,NaN,1
39,NaN,1


In [170]:
#there are 248 columns which can be filled as 0
cust_demo.loc[(cust_demo['no_of_children'].isnull()) & (cust_demo['family_size'] == '1'),'no_of_children']='0'

In [171]:
cust_demo.loc[(cust_demo['no_of_children'].isnull()) & (cust_demo['family_size'] == '1')][['no_of_children','family_size']]

,no_of_children,family_size


In [172]:
# follow the same process for marital_status as well.
cust_demo.loc[(cust_demo['marital_status'].isnull()) & (cust_demo['family_size'] == '1')][['marital_status','family_size']]

,marital_status,family_size
10,NaN,1
11,NaN,1
15,NaN,1
29,NaN,1
31,NaN,1
33,NaN,1
39,NaN,1
44,NaN,1
47,NaN,1
51,NaN,1


In [173]:
cust_demo.loc[(cust_demo['family_size'] == '1') & (cust_demo['marital_status'] == "Married")][['marital_status','family_size']]

,marital_status,family_size


In [174]:
#So there are no customer is classified as married if the family size is single, so we can classify all the customers whose
# family size is 1 to marital status as Single
cust_demo.loc[(cust_demo['marital_status'].isnull()) & (cust_demo['family_size'] == '1'),'marital_status']='Single'
cust_demo.loc[(cust_demo['marital_status'].isnull()) & (cust_demo['family_size'] == '1')][['marital_status','family_size']]

,marital_status,family_size


In [175]:
#Now let's check how many null values in customer dataset
for col in cust_demo.columns:
    print("{} has {} null values".format(col,cust_demo[col].isnull().sum()))

customer_id has 0 null values
age_range has 0 null values
marital_status has 146 null values
rented has 0 null values
family_size has 0 null values
no_of_children has 290 null values
income_bracket has 0 null values


In [186]:
# There are still few null values in no_of_children,marital_status let's try to fill these values
# Let's see the other fields when the marital_status is null
for col in ['age_range','family_size']:
    print(col,"\n===========")
    print(cust_demo.loc[cust_demo['marital_status'].isnull()][col].value_counts())

age_range 
46-55    54
26-35    37
36-45    29
70+      11
18-25    10
56-70     5
Name: age_range, dtype: int64
family_size 
2     97
3     30
4     13
5+     6
Name: family_size, dtype: int64


In [239]:
# So will take the mode of the combination of age_range&family_size  fields as they seem to be related

newcust_df=cust_demo.groupby(['age_range','family_size']).agg({'marital_status':lambda x: x.mode()[0] if len(x.mode()) == 2 else x.mode()}).reset_index()


,age_range,family_size,marital_status
0,18-25,1,Single
1,18-25,2,Married
2,18-25,3,Married
3,18-25,4,Single
4,18-25,5+,Single
5,26-35,1,Single
6,26-35,2,Married
7,26-35,3,Married
8,26-35,4,Married
9,26-35,5+,Married


In [240]:
for i,row in newcust_df.iterrows():
    cust_demo.loc[(cust_demo['marital_status'].isnull())&(cust_demo['age_range'] == row['age_range'])&(cust_demo['family_size']==row['family_size']),'marital_status']=row['marital_status']
    
cust_demo.head()

age_range          18-25
family_size            1
marital_status    Single
Name: 0, dtype: object
age_range           18-25
family_size             2
marital_status    Married
Name: 1, dtype: object
age_range           18-25
family_size             3
marital_status    Married
Name: 2, dtype: object
age_range          18-25
family_size            4
marital_status    Single
Name: 3, dtype: object
age_range          18-25
family_size           5+
marital_status    Single
Name: 4, dtype: object
age_range          26-35
family_size            1
marital_status    Single
Name: 5, dtype: object
age_range           26-35
family_size             2
marital_status    Married
Name: 6, dtype: object
age_range           26-35
family_size             3
marital_status    Married
Name: 7, dtype: object
age_range           26-35
family_size             4
marital_status    Married
Name: 8, dtype: object
age_range           26-35
family_size            5+
marital_status    Married
Name: 9, dtype: object
ag

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,NaN,4
1,6,46-55,Married,0,2,NaN,5
2,7,26-35,Married,0,3,1,3
3,8,26-35,Married,0,4,2,6
4,10,46-55,Single,0,1,0,5


In [243]:
#If the person is married and family_size is 2 then the no_of_children should be 0
# So write a condition to filter family_size=2 & marital_status=married
cust_demo.loc[(cust_demo['marital_status']=="Married")&(cust_demo['family_size'] == '2'),'no_of_children']='0'
cust_demo['no_of_children'].isnull().sum()

26

In [244]:
#so there are still 26 records which have null values, let's see what are those

cust_demo[cust_demo['no_of_children'].isnull()]


,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
5,11,70+,Single,0,2,NaN,1
12,22,36-45,Single,0,2,NaN,4
20,38,46-55,Single,0,2,NaN,5
23,41,46-55,Single,0,2,NaN,4
42,82,70+,Single,0,2,NaN,4
46,87,46-55,Single,0,2,NaN,4
69,136,56-70,Single,0,2,NaN,1
132,265,18-25,Single,0,2,NaN,3
147,292,26-35,Single,1,2,NaN,1
149,294,18-25,Single,0,2,NaN,5


In [253]:
#I am assuming that if the person's age range is greater than 26-35 
#and the marrital_status is Single he would probably married and divorced, 
# so i am assigning the no_of_children as 1 since the family size is 2 for these
cust_demo.loc[((cust_demo['age_range']).isin(['18-25','26-35'])) & (cust_demo['no_of_children'].isnull()),'no_of_children']='0'
cust_demo.loc[((cust_demo['age_range']).isin(['36-45','46-55','56-70','70+'])) & (cust_demo['no_of_children'].isnull()),'no_of_children']='1'

In [255]:
for col in cust_demo.columns:
    print('{} has {} null values'.format(col,cust_demo[col].isnull().sum()))

customer_id has 0 null values
age_range has 0 null values
marital_status has 0 null values
rented has 0 null values
family_size has 0 null values
no_of_children has 0 null values
income_bracket has 0 null values


In [256]:
#Finally we have get rid of the null values
#Let's go ahead and join the datasets now.

In [257]:
train.shape

(78369, 5)

In [306]:
newtrain=pd.merge(train,campaign,on="campaign_id",how="left")
newtrain.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date
0,1,13,27,1053,0,X,19/05/13,05/07/13
1,2,13,116,48,0,X,19/05/13,05/07/13
2,6,9,635,205,0,Y,11/03/13,12/04/13
3,7,13,644,1050,0,X,19/05/13,05/07/13
4,9,8,1017,1489,0,X,16/02/13,05/04/13


In [307]:
newtrain.shape

(78369, 8)

In [308]:
newtrain=newtrain.merge(cust_demo,on="customer_id",how="left")

In [309]:
newtrain.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,13,27,1053,0,X,19/05/13,05/07/13,46-55,Single,0.0,1,0,5.0
1,2,13,116,48,0,X,19/05/13,05/07/13,36-45,Married,0.0,2,0,3.0
2,6,9,635,205,0,Y,11/03/13,12/04/13,46-55,Married,0.0,2,0,7.0
3,7,13,644,1050,0,X,19/05/13,05/07/13,NaN,NaN,NaN,NaN,NaN,NaN
4,9,8,1017,1489,0,X,16/02/13,05/04/13,46-55,Married,0.0,2,0,3.0


In [310]:
for col in newtrain.columns:
    print("{} has {} null values".format(col,newtrain[col].isnull().sum()))

id has 0 null values
campaign_id has 0 null values
coupon_id has 0 null values
customer_id has 0 null values
redemption_status has 0 null values
campaign_type has 0 null values
start_date has 0 null values
end_date has 0 null values
age_range has 34708 null values
marital_status has 34708 null values
rented has 34708 null values
family_size has 34708 null values
no_of_children has 34708 null values
income_bracket has 34708 null values


In [311]:
#Even after filling the null values in cust_demo table, we still have null values after merging it with train table
#which means there are customer ids present in train which are not present in cust_demo
# let's see the unique customers for whom have these null values
newtrain.loc[newtrain['age_range'].isnull()]['customer_id'].value_counts()

63      164
787     154
1140    138
432     136
1549    136
1429    132
1044    130
1215    130
413     128
659     127
998     125
1014    125
23      119
183     117
709     116
1372    116
1555    115
1007    115
711     114
484     113
1012    111
1029    111
1157    109
1310    105
437     105
88      104
240     103
435     101
633     101
1418    100
       ... 
1423     15
846      15
1325     15
1307     14
1251     14
1283     14
70       14
1501     14
1575     14
137      14
1207     14
845      14
1564     13
1508     13
255      13
547      13
120      13
1160     13
187      13
1248     12
1513     12
477      11
1511     11
160      11
472      11
109      11
1476      8
219       8
618       8
1354      8
Name: customer_id, Length: 725, dtype: int64

In [ ]:
#So there are about 725 customers which have the null values or doesn't present on the cust_demo table

In [312]:
newtrain.loc[newtrain['family_size'].isnull()]

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,age_range,marital_status,rented,family_size,no_of_children,income_bracket
3,7,13,644,1050,0,X,19/05/13,05/07/13,NaN,NaN,NaN,NaN,NaN,NaN
8,17,30,857,523,0,X,19/11/12,04/01/13,NaN,NaN,NaN,NaN,NaN,NaN
14,24,5,893,761,0,Y,12/01/13,15/02/13,NaN,NaN,NaN,NaN,NaN,NaN
21,32,30,320,1255,0,X,19/11/12,04/01/13,NaN,NaN,NaN,NaN,NaN,NaN
25,39,26,326,63,0,X,12/08/12,21/09/12,NaN,NaN,NaN,NaN,NaN,NaN
26,42,30,363,88,0,X,19/11/12,04/01/13,NaN,NaN,NaN,NaN,NaN,NaN
28,45,8,383,1095,0,X,16/02/13,05/04/13,NaN,NaN,NaN,NaN,NaN,NaN
29,46,13,482,1096,0,X,19/05/13,05/07/13,NaN,NaN,NaN,NaN,NaN,NaN
30,48,26,813,919,0,X,12/08/12,21/09/12,NaN,NaN,NaN,NaN,NaN,NaN
34,55,30,500,91,0,X,19/11/12,04/01/13,NaN,NaN,NaN,NaN,NaN,NaN


In [313]:
ci_item_join=ci_mapping.merge(item_data,on="item_id",how="inner")
ci_item_join.head()

,coupon_id,item_id,brand,brand_type,category
0,105,37,56,Local,Grocery
1,6,37,56,Local,Grocery
2,22,37,56,Local,Grocery
3,31,37,56,Local,Grocery
4,107,75,56,Local,Grocery


In [314]:
ci_item_groupby=ci_item_join.groupby('coupon_id').agg({'brand_type':lambda x: x.mode(),
                                       'category':lambda x: x.mode(),
                                       'brand': lambda x: x.mode(),
                                       'item_id':lambda x: int(statistics.median(list(x.mode())))})
ci_item_groupby.head()

,brand_type,category,brand,item_id
coupon_id,,,,
1,Established,Natural Products,1475,51027
2,Established,Grocery,2084,7741
3,Established,Grocery,278,59203
4,Established,Grocery,544,51241
5,Established,Pharmaceutical,5357,53496


In [315]:
ci_item_groupby.reset_index(inplace=True)

In [316]:
newtrain=newtrain.merge(ci_item_groupby,on="coupon_id",how="inner")
newtrain.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,age_range,marital_status,rented,family_size,no_of_children,income_bracket,brand_type,category,brand,item_id
0,1,13,27,1053,0,X,19/05/13,05/07/13,46-55,Single,0.0,1,0,5.0,Established,Grocery,1105,25083
1,370,13,27,1168,0,X,19/05/13,05/07/13,18-25,Married,1.0,2,0,4.0,Established,Grocery,1105,25083
2,554,13,27,1061,0,X,19/05/13,05/07/13,36-45,Single,0.0,1,0,3.0,Established,Grocery,1105,25083
3,1958,13,27,1240,0,X,19/05/13,05/07/13,26-35,Married,0.0,5+,3+,5.0,Established,Grocery,1105,25083
4,2075,13,27,351,0,X,19/05/13,05/07/13,NaN,NaN,NaN,NaN,NaN,NaN,Established,Grocery,1105,25083


In [270]:
ct_data.shape

(1324566, 7)

In [317]:
ct_item_merged=ct_data.merge(item_data,on="item_id",how="inner")
ct_item_merged.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0,56,Local,Natural Products
1,2012-01-02,135,26830,1,35.26,-10.69,0.0,56,Local,Natural Products
2,2012-01-02,464,26830,1,35.26,-10.69,0.0,56,Local,Natural Products
3,2012-01-12,1185,26830,1,45.95,0.00,0.0,56,Local,Natural Products
4,2012-01-13,1140,26830,1,45.95,0.00,0.0,56,Local,Natural Products


In [318]:
ci_item_groupby=ct_item_merged.groupby('customer_id').agg({'item_id': lambda x:Counter(x).most_common()[0][0],
                                           'quantity':'max',
                                           'selling_price':'mean',
                                           'other_discount':'mean',
                                           'coupon_discount':'mean',
                                           'brand':lambda x: Counter(x).most_common()[0][0],
                                           'brand_type':lambda x: Counter(x).most_common()[0][0],
                                           'category':lambda x: Counter(x).most_common()[0][0]
                                          }).reset_index()

In [319]:
ci_item_groupby.head(n=10)

,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category
0,1,8307,5,94.001842,-16.250382,-2.019876,56,Established,Grocery
1,2,34047,5,102.864033,-16.830430,-0.595084,56,Established,Grocery
2,3,32573,6949,103.617404,-22.714227,-3.091546,56,Established,Grocery
3,4,47968,5,154.423727,-13.305409,-0.404773,56,Established,Grocery
4,5,35304,14638,130.827146,-13.657917,-0.114684,56,Established,Grocery
5,6,35630,4,102.072419,-12.001938,-0.702607,56,Established,Grocery
6,7,20057,13552,102.067455,-17.803799,-0.745897,56,Established,Grocery
7,8,49009,31721,227.224697,-23.528945,-5.011601,56,Established,Grocery
8,9,49009,10757,102.829210,-21.101885,-0.808312,56,Established,Grocery
9,10,43641,11613,103.788798,-11.479552,0.000000,56,Established,Grocery


In [349]:
newtrain1=newtrain.merge(ci_item_groupby,on=["customer_id"],how="left")

In [350]:
newtrain1.head(n=10)

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,age_range,marital_status,...,brand_x,item_id_x,item_id_y,quantity,selling_price,other_discount,coupon_discount,brand_y,brand_type_y,category_y
0,1,13,27,1053,0,X,19/05/13,05/07/13,46-55,Single,...,1105,25083,14458,12444,184.260484,-33.168935,-0.287258,56,Established,Grocery
1,370,13,27,1168,0,X,19/05/13,05/07/13,18-25,Married,...,1105,25083,45069,12707,83.809140,-21.016867,-1.447444,56,Established,Grocery
2,554,13,27,1061,0,X,19/05/13,05/07/13,36-45,Single,...,1105,25083,64193,10,109.322227,-16.092983,-0.976282,56,Established,Grocery
3,1958,13,27,1240,0,X,19/05/13,05/07/13,26-35,Married,...,1105,25083,31662,6,93.595770,-11.163103,-0.264591,56,Established,Grocery
4,2075,13,27,351,0,X,19/05/13,05/07/13,NaN,NaN,...,1105,25083,7354,7627,73.584467,-13.297944,-0.248629,56,Established,Grocery
5,2496,13,27,1531,0,X,19/05/13,05/07/13,NaN,NaN,...,1105,25083,14497,32,101.195707,-11.407690,0.000000,56,Established,Grocery
6,3062,13,27,810,0,X,19/05/13,05/07/13,NaN,NaN,...,1105,25083,12634,8337,78.669005,-10.589288,0.000000,56,Established,Grocery
7,3801,13,27,1507,0,X,19/05/13,05/07/13,NaN,NaN,...,1105,25083,24273,16130,67.658147,-16.195415,-0.440974,56,Established,Grocery
8,4494,13,27,1093,0,X,19/05/13,05/07/13,46-55,Married,...,1105,25083,45069,24583,135.754971,-16.593945,-0.655391,56,Established,Grocery
9,7266,13,27,111,0,X,19/05/13,05/07/13,NaN,NaN,...,1105,25083,35630,5,123.979981,-11.669702,0.000000,56,Established,Grocery


In [351]:
newtrain1['start_date']=newtrain1['start_date'].apply(lambda x: x.split('/')[1]+"/"+x.split('/')[0]+"/"+x.split('/')[-1])
newtrain1['end_date']=newtrain1['end_date'].apply(lambda x: x.split('/')[1]+"/"+x.split('/')[0]+"/"+x.split('/')[-1])
#pd.to_datetime(newtrain1['end_date'])-pd.to_datetime(newtrain1['start_date'])

In [353]:
newtrain1['coupon_campaign_days']=pd.to_datetime(newtrain1['end_date'])-pd.to_datetime(newtrain1['start_date'])

In [361]:
newtrain1['coupon_campaign_days']=newtrain1['coupon_campaign_days'].apply(lambda x: x.days)


AttributeError: 'int' object has no attribute 'days'

In [364]:
newtrain1.columns

Index(['id', 'campaign_id', 'coupon_id', 'customer_id', 'redemption_status',
       'campaign_type', 'start_date', 'end_date', 'age_range',
       'marital_status', 'rented', 'family_size', 'no_of_children',
       'income_bracket', 'brand_type_x', 'category_x', 'brand_x', 'item_id_x',
       'item_id_y', 'quantity', 'selling_price', 'other_discount',
       'coupon_discount', 'brand_y', 'brand_type_y', 'category_y',
       'coupon_campaign_days'],
      dtype='object')

In [367]:
newtrain2=newtrain1.drop(['id','campaign_id','coupon_id','customer_id','start_date','end_date','age_range','marital_status',
                         'rented','family_size','no_of_children', 'income_bracket', 'brand_type_x',
                          'category_x', 'brand_x', 'item_id_x', 'item_id_y' ],axis=1)
newtrain2.head()

,redemption_status,campaign_type,quantity,selling_price,other_discount,coupon_discount,brand_y,brand_type_y,category_y,coupon_campaign_days
0,0,X,12444,184.260484,-33.168935,-0.287258,56,Established,Grocery,47
1,0,X,12707,83.809140,-21.016867,-1.447444,56,Established,Grocery,47
2,0,X,10,109.322227,-16.092983,-0.976282,56,Established,Grocery,47
3,0,X,6,93.595770,-11.163103,-0.264591,56,Established,Grocery,47
4,0,X,7627,73.584467,-13.297944,-0.248629,56,Established,Grocery,47


In [377]:
newtrain3=pd.get_dummies(newtrain2,columns=['campaign_type','brand_type_y','category_y','brand_y'])

In [378]:
newtrain3.head(n=8)

,redemption_status,quantity,selling_price,other_discount,coupon_discount,coupon_campaign_days,campaign_type_X,campaign_type_Y,brand_type_y_Established,brand_type_y_Local,...,brand_y_1337,brand_y_1447,brand_y_1562,brand_y_1564,brand_y_1996,brand_y_2025,brand_y_2084,brand_y_2370,brand_y_2379,brand_y_4991
0,0,12444,184.260484,-33.168935,-0.287258,47,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,12707,83.809140,-21.016867,-1.447444,47,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,10,109.322227,-16.092983,-0.976282,47,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,6,93.595770,-11.163103,-0.264591,47,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,7627,73.584467,-13.297944,-0.248629,47,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,0,32,101.195707,-11.407690,0.000000,47,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,0,8337,78.669005,-10.589288,0.000000,47,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,0,16130,67.658147,-16.195415,-0.440974,47,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [381]:
X=newtrain3.drop('redemption_status',axis=1)
y=newtrain3['redemption_status']
train_X,test_x,train_Y,test_y=train_test_split(X,y,test_size=0.3,random_state=42)

In [382]:
lgbc = lgb.LGBMClassifier(n_estimators=1000, max_depth=5, learning_rate=0.01, random_state=i, colsample_bytree=0.2, reg_lambda=15, reg_alpha=10)
lgbc.fit(train_X,train_Y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.2,
               importance_type='split', learning_rate=0.01, max_depth=5,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1000, n_jobs=-1, num_leaves=31, objective=None,
               random_state=27, reg_alpha=10, reg_lambda=15, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [395]:
preds=lgbc.predict(test_x)

In [396]:
roc_auc_score(test_y,preds)

0.5

In [397]:
lgbc.feature_importances_

array([1217, 1558, 1306, 1150,  452,  136,  158,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0])

In [438]:
param_lgb = { 'boosting_type': 'gbdt', 'objective': 'binary', 'metric':'auc',
             'bagging_freq':1, 'subsample':1, 'feature_fraction': 0.7,
              'num_leaves': 8, 'learning_rate': 0.05, 'lambda_l1':5,'max_bin':255}
# form LightGBM datasets
dtrain_lgb = lgb.Dataset(train_X, label=train_Y)
# LightGBM, cross-validation
cv_result_lgb = lgb.cv(param_lgb, 
                       dtrain_lgb, 
                       num_boost_round=1000, 
                       nfold=10, 
                       stratified=True, 
                       early_stopping_rounds=150, 
                       verbose_eval=100, 
                       show_stdv=True)

[100]	cv_agg's auc: 0.85592 + 0.0209212
[200]	cv_agg's auc: 0.860141 + 0.0209412
[300]	cv_agg's auc: 0.862533 + 0.0200626
[400]	cv_agg's auc: 0.862505 + 0.0200074


In [439]:

model_lgb = lgb.train(param_lgb, dtrain_lgb, num_boost_round=10000)
y_pred=model_lgb.predict(test_x)

In [440]:
roc_auc_score(test_y,y_pred)
#preds
#model_lgb.feature_importance()

0.8672002334180651

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
from tqdm import tqdm_notebook as tqdm
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from matplotlib import pyplot as plt
import seaborn as sns
from collections import defaultdict, Counter
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures, RobustScaler
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score as auc
from sklearn.linear_model import LogisticRegression
from scipy.special import logit
file = {
	'test' : '../input/amexpert-2019/test.csv',
	'train':'../input/amexpert-2019/train.csv',
	'submission':'../input/amexpert-2019/submission.csv',
	'coupon_item_mapping' :'../input/amexpert-2019/coupon_item_mapping.csv',
	'campaign_data' : '../input/amexpert-2019/campaign_data.csv',
	'item_data' : '../input/amexpert-2019/item_data.csv',
	'customer_transaction_data':'../input/amexpert-2019/customer_transaction_data.csv',
	'customer_demographics':'../input/amexpert-2019/customer_demographics.csv',
}
#campaign_data, customer_demographics customer_transaction_data
# item_data, coupon_item_mapping
train = pd.read_csv(file.get("train"))#
test = pd.read_csv(file.get("test"))#
​
coupon_item_mapping = pd.read_csv(file.get("coupon_item_mapping"))#No
item_data = pd.read_csv(file.get("item_data"))# may be yes
customer_transaction_data = pd.read_csv(file.get("customer_transaction_data"))#may be yes 
​
campaign_data = pd.read_csv(file.get("campaign_data"))#
customer_demographics = pd.read_csv(file.get("customer_demographics"))#
submission = pd.read_csv(file.get("submission"))
data = pd.concat([train, test], sort=False).reset_index(drop = True)
ltr = len(train)
data = data.merge(campaign_data, on='campaign_id')#  campaign_data
data['start_date'] = pd.to_datetime(data['start_date'], dayfirst=True)
data['end_date'] = pd.to_datetime(data['end_date'], dayfirst=True)
data['campaign_type'] = pd.Series(data['campaign_type'].factorize()[0]).replace(-1, np.nan)
#######################################################################
# customer_demographics
# customer_demographics
customer_demographics['no_of_children'] = customer_demographics['no_of_children'].replace('3+', 3).astype(float)
customer_demographics['family_size'] = customer_demographics['family_size'].replace('5+', 3).astype(float)
customer_demographics['marital_status'] = pd.Series(customer_demographics['marital_status'].factorize()[0]).replace(-1, np.nan)
customer_demographics['age_range'] = pd.Series(customer_demographics['age_range'].factorize()[0]).replace(-1, np.nan)
​
# rented
rented_mean = customer_demographics.groupby("customer_id")['rented'].mean().to_dict()
data['rented_mean'] = data['customer_id'].map(rented_mean)
# income_bracket
income_bracket_sum = customer_demographics.groupby("customer_id")['income_bracket'].sum().to_dict()
data['income_bracket_sum'] = data['customer_id'].map(income_bracket_sum)
# age_range
age_range_mean = customer_demographics.groupby("customer_id")['age_range'].mean().to_dict()
data['age_range_mean'] = data['customer_id'].map(age_range_mean)
# family_size
family_size_mean = customer_demographics.groupby("customer_id")['family_size'].mean().to_dict()
data['family_size_mean'] = data['customer_id'].map(family_size_mean)
# no_of_children
no_of_children_mean = customer_demographics.groupby("customer_id")['no_of_children'].mean().to_dict()
data['no_of_children_mean'] = data['customer_id'].map(no_of_children_mean)
no_of_children_count = customer_demographics.groupby("customer_id")['no_of_children'].count().to_dict()
data['no_of_children_count'] = data['customer_id'].map(no_of_children_count)
# marital_status
marital_status_count = customer_demographics.groupby("customer_id")['marital_status'].count().to_dict()
data['marital_status_count'] = data['customer_id'].map(marital_status_count)
#############################################################################
# customer_transaction_data
customer_transaction_data['date'] = pd.to_datetime(customer_transaction_data['date'])
# quantity	
quantity_mean = customer_transaction_data.groupby("customer_id")['quantity'].mean().to_dict()
data['quantity_mean'] = data['customer_id'].map(quantity_mean)
#coupon_discount
coupon_discount_mean = customer_transaction_data.groupby("customer_id")['coupon_discount'].mean().to_dict()
data['coupon_discount_mean'] = data['customer_id'].map(coupon_discount_mean)
# other_discount
other_discount_mean = customer_transaction_data.groupby("customer_id")['other_discount'].mean().to_dict()
data['other_discount_mean'] = data['customer_id'].map(other_discount_mean)
# day
customer_transaction_data['day'] = customer_transaction_data.date.dt.day
date_day_mean = customer_transaction_data.groupby("customer_id")['day'].mean().to_dict()
data['date_day_mean'] = data['customer_id'].map(date_day_mean)
#coupon_item_mapping, item_data
coupon_item_mapping = coupon_item_mapping.merge(item_data, how = 'left', on = 'item_id')
coupon_item_mapping['brand_type'] = pd.Series(coupon_item_mapping['brand_type'].factorize()[0]).replace(-1, np.nan)
coupon_item_mapping['category'] = pd.Series(coupon_item_mapping['category'].factorize()[0]).replace(-1, np.nan)
​
category = coupon_item_mapping.groupby("coupon_id")['category'].mean().to_dict()
data['category_mean'] = data['coupon_id'].map(category)
category = coupon_item_mapping.groupby("coupon_id")['category'].count().to_dict()
data['category_count'] = data['coupon_id'].map(category)
category = coupon_item_mapping.groupby("coupon_id")['category'].nunique().to_dict()
data['category_nunique'] = data['coupon_id'].map(category)
category = coupon_item_mapping.groupby("coupon_id")['category'].max().to_dict()
data['category_max'] = data['coupon_id'].map(category)
category = coupon_item_mapping.groupby("coupon_id")['category'].min().to_dict()
data['category_min'] = data['coupon_id'].map(category)
​
brand_mean = coupon_item_mapping.groupby("coupon_id")['brand'].mean().to_dict()
data['brand_mean'] = data['coupon_id'].map(brand_mean)
brand_mean = coupon_item_mapping.groupby("coupon_id")['brand'].count().to_dict()
data['brand_count'] = data['coupon_id'].map(brand_mean)
brand_mean = coupon_item_mapping.groupby("coupon_id")['brand'].min().to_dict()
data['brand_min'] = data['coupon_id'].map(brand_mean)
brand_mean = coupon_item_mapping.groupby("coupon_id")['brand'].max().to_dict()
data['brand_max'] = data['coupon_id'].map(brand_mean)
brand_mean = coupon_item_mapping.groupby("coupon_id")['brand'].nunique().to_dict()
data['brand_nunique'] = data['coupon_id'].map(brand_mean)
​
# selling_price
selling_price_mean = customer_transaction_data.groupby("customer_id")['selling_price'].mean().to_dict()
data['selling_price_mean'] = data['customer_id'].map(selling_price_mean)
selling_price_mean = customer_transaction_data.groupby("customer_id")['selling_price'].sum().to_dict()
data['selling_price_sum'] = data['customer_id'].map(selling_price_mean)
selling_price_mean = customer_transaction_data.groupby("customer_id")['selling_price'].min().to_dict()
data['selling_price_min'] = data['customer_id'].map(selling_price_mean)
selling_price_mean = customer_transaction_data.groupby("customer_id")['selling_price'].max().to_dict()
data['selling_price_max'] = data['customer_id'].map(selling_price_mean)
selling_price_mean = customer_transaction_data.groupby("customer_id")['selling_price'].nunique().to_dict()
data['selling_price_nunique'] = data['customer_id'].map(selling_price_mean)
train_cols = [i for i in data.columns if i not in ['id','redemption_status','start_date','end_date']]
train_cols = ['campaign_id','coupon_id','campaign_type','rented_mean','income_bracket_sum','age_range_mean','family_size_mean',
 'no_of_children_mean',
 'no_of_children_count',
 'marital_status_count',
 'quantity_mean',
 'coupon_discount_mean',
 'other_discount_mean',
 'date_day_mean',
 'category_mean',
 'category_nunique',
 'category_max',
 'category_min',
 'brand_mean',
 'brand_max',
 'brand_nunique',
 'selling_price_mean',
 'selling_price_min',
 'selling_price_nunique']
train = data[data['redemption_status'].notnull()]
test = data[data['redemption_status'].isnull()]
dummies = pd.get_dummies(data[train_cols].fillna(0), columns=train_cols, drop_first=True, sparse=True)
train_ohe = dummies.iloc[:train.shape[0], :]
test_ohe = dummies.iloc[train.shape[0]:, :]
​
print(train_ohe.shape)
print(test_ohe.shape)
train_ohe = train_ohe.sparse.to_coo().tocsr()
test_ohe = test_ohe.sparse.to_coo().tocsr()
# Model
def run_cv_model(train, test, target, model_fn, params={}, eval_fn=None, label='model'):
    kf = StratifiedKFold(n_splits=10, shuffle = True, random_state = 228)
    fold_splits = kf.split(train, target)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros((train.shape[0]))
    i = 1
    for dev_index, val_index in fold_splits:
        print('Started ' + label + ' fold ' + str(i) + '/5')
        dev_X, val_X = train[dev_index], train[val_index]
        dev_y, val_y = target[dev_index], target[val_index]
        params2 = params.copy()
        pred_val_y, pred_test_y = model_fn(dev_X, dev_y, val_X, val_y, test, params2)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index] = pred_val_y
        if eval_fn is not None:
            cv_score = eval_fn(val_y, pred_val_y)
            cv_scores.append(cv_score)
            print(label + ' cv score {}: {}'.format(i, cv_score))
        i += 1
    print('{} cv scores : {}'.format(label, cv_scores))
    print('{} cv mean score : {}'.format(label, np.mean(cv_scores)))
    print('{} cv std score : {}'.format(label, np.std(cv_scores)))
    pred_full_test = pred_full_test / 10.0
    results = {'label': label,
              'train': pred_train, 'test': pred_full_test,
              'cv': cv_scores}
    return results
​
​
def runLR(train_X, train_y, test_X, test_y, test_X2, params):
    print('Train LR')
    model = LogisticRegression(**params)
    model.fit(train_X, train_y)
    print('Predict 1/2')
    pred_test_y = logit(model.predict_proba(test_X)[:, 1])
    print('Predict 2/2')
    pred_test_y2 = logit(model.predict_proba(test_X2)[:, 1])
    return pred_test_y, pred_test_y2
​
target = train['redemption_status'].values
lr_params = {'solver': 'lbfgs','C': 1.8,'max_iter' : 2000}
results = run_cv_model(train_ohe, test_ohe, target, runLR, lr_params, auc, 'lr')
day = 2
sub = 3
name = f"day_{day}_sub_{sub}"
tmp = dict(zip(test.id.values, results['test']))
answer1 = pd.DataFrame()
answer1['id'] = test.id.values
answer1['redemption_status'] = answer1['id'].map(tmp)
answer1.to_csv(f'{name}.csv', index = None)